In [2]:
import pandas as pd
import json

# Create top 10 per country 

In [14]:
df = pd.read_csv('data/rankings.csv')

In [15]:
df.date = pd.to_datetime(df.date)
df['year'] = df['date'].dt.year


In [6]:
df.head()

,artist_id,country,date,chartType,artist_name,image_link,currentRank,previousRank,peakRank,peakDate,appearancesOnChart,consecutiveAppearancesOnChart,entryStatus,entryRank,entryDate,year
0,4q3ewBCX7sLwd24euuV69X,GLOBAL,2025-05-15,TOP_ARTIST,Bad Bunny,https://i.scdn.co/image/ab6761610000517481f47f...,1,1,1,2021-10-28,187,187,NO_CHANGE,2,2021-10-21,2025
1,06HL4z0CvFAxyc27GXpf02,GLOBAL,2025-05-15,TOP_ARTIST,Taylor Swift,https://i.scdn.co/image/ab67616100005174e672b5...,2,2,1,2021-11-11,187,187,NO_CHANGE,6,2021-10-21,2025
2,1Xyo4u8uXC1ZmMpatF05PJ,GLOBAL,2025-05-15,TOP_ARTIST,The Weeknd,https://i.scdn.co/image/ab676161000051749e5289...,3,4,1,2022-01-13,187,187,MOVED_UP,11,2021-10-21,2025
3,3TVXtAsR1Inumwj472S9r4,GLOBAL,2025-05-15,TOP_ARTIST,Drake,https://i.scdn.co/image/ab67616100005174429338...,4,3,1,2023-10-12,187,187,MOVED_DOWN,3,2021-10-21,2025
4,0ys2OFYzWYB5hRDLCsBqxt,GLOBAL,2025-05-15,TOP_ARTIST,Fuerza Regida,https://i.scdn.co/image/ab67616100005174158a3d...,5,7,5,2025-05-15,128,128,MOVED_UP,190,2022-12-08,2025


In [7]:
def compute_ranking(artistDf):
    rank = artistDf.currentRank.sum()
    return rank

In [16]:
result = {}


grouped = df.groupby(['year', 'country'])

for (year, country, ), group in grouped:
    bySong = group.groupby('artist_name')
    year = int(year)

    ranksToTrack = {}
    trackToRank = {}
    trackRow = {}

    for track, trackDf in bySong:
        rank = compute_ranking(trackDf)

        if rank in ranksToTrack:
            ranksToTrack[rank].append(track)
        else:   
            ranksToTrack[rank] = [track]

        trackToRank[track] = int(rank)
        trackRow[track] = trackDf.iloc[0]

    ranksSorted = sorted(ranksToTrack.keys())

    sortedTracks = []
    for rank in ranksSorted[:10]:
        if len(sortedTracks) >= 10:
            break
        artists = ranksToTrack[rank]
        for track in artists:
            if len(sortedTracks) >= 10:
                break
            sortedTracks.append(track)

    if year not in result:
        result[year] = {}
    if country not in result[year]:
        result[year][country] = {}
    for i, track in enumerate(sortedTracks):
        rank_key = int(i+1)
        if rank_key not in result[year][country]:
            result[year][country][rank_key] = {}
        result[year][country][rank_key]['name'] = track
        result[year][country][rank_key]['aggregated_rank'] = trackToRank[track]
        if pd.notna(trackRow[track].image_link):
            result[year][country][rank_key]['image'] = trackRow[track].image_link
        else :
            result[year][country][rank_key]['image'] = None

with open('top10_artist_by_country.json', 'w') as f:
    json.dump(result, f, indent=2, )

# Top 10 song per year

In [3]:
df = pd.read_csv('data/song.csv')

In [4]:
df.date = pd.to_datetime(df.date)
df['year'] = df['date'].dt.year


In [5]:
df.head()

,currentRank,previousRank,peakRank,peakDate,appearancesOnChart,consecutiveAppearancesOnChart,rankingMetric,entryStatus,entryRank,entryDate,...,country,date,chartType,track_name,artists,labels,image_link,metricValue,metricType,year
0,1,2,1,2025-05-15,14,14,"{'value': '40299624', 'type': 'STREAMS'}",MOVED_UP,60,2025-02-13,...,GLOBAL,2025-05-15,TOP_TRACK,Ordinary,['Alex Warren'],['Atlantic Records'],https://i.scdn.co/image/ab67616d00001e02164871...,40299624,STREAMS,2025
1,2,1,1,2024-08-29,39,39,"{'value': '38399066', 'type': 'STREAMS'}",MOVED_DOWN,3,2024-08-22,...,GLOBAL,2025-05-15,TOP_TRACK,Die With A Smile,"['Lady Gaga', 'Bruno Mars']",['Interscope Records'],https://i.scdn.co/image/ab67616d00001e02b0860c...,38399066,STREAMS,2025
2,3,3,1,2024-07-18,52,52,"{'value': '34553703', 'type': 'STREAMS'}",NO_CHANGE,7,2024-05-23,...,GLOBAL,2025-05-15,TOP_TRACK,BIRDS OF A FEATHER,['Billie Eilish'],['Darkroom/Interscope Records'],https://i.scdn.co/image/ab67616d00001e0271d62e...,34553703,STREAMS,2025
3,4,4,1,2024-10-24,30,30,"{'value': '27951926', 'type': 'STREAMS'}",NO_CHANGE,1,2024-10-24,...,GLOBAL,2025-05-15,TOP_TRACK,APT.,"['ROSÉ', 'Bruno Mars']",['Atlantic Records'],https://i.scdn.co/image/ab67616d00001e025074bd...,27951926,STREAMS,2025
4,5,9,1,2025-01-16,19,19,"{'value': '27324024', 'type': 'STREAMS'}",MOVED_UP,15,2025-01-09,...,GLOBAL,2025-05-15,TOP_TRACK,DtMF,['Bad Bunny'],['Rimas Entertainment LLC.'],https://i.scdn.co/image/ab67616d00001e02bbd45c...,27324024,STREAMS,2025


In [12]:
result = {}
grouped = df.groupby(['year', 'country'])

for (year, country, ), group in grouped:
    bySong = group.groupby('track_name')
    year = int(year)

    ranksToTrack = {}
    trackToRank = {}
    trackRow = {}

    for track, trackDf in bySong:
        rank = compute_ranking(trackDf)

        if rank in ranksToTrack:
            ranksToTrack[rank].append(track)
        else:   
            ranksToTrack[rank] = [track]

        trackToRank[track] = int(rank)
        trackRow[track] = trackDf.iloc[0]

    ranksSorted = sorted(ranksToTrack.keys())

    sortedTracks = []
    for rank in ranksSorted[:10]:
        if len(sortedTracks) >= 10:
            break
        tracks = ranksToTrack[rank]
        for track in tracks:
            if len(sortedTracks) >= 10:
                break
            sortedTracks.append(track)

    if year not in result:
        result[year] = {}
    if country not in result[year]:
        result[year][country] = {}
    for i, track in enumerate(sortedTracks):
        rank_key = int(i+1)
        if rank_key not in result[year][country]:
            result[year][country][rank_key] = {}
        result[year][country][rank_key]['name'] = f"{track}, {trackRow[track].artists.replace("[", "").split("'")[1]}"
        result[year][country][rank_key]['aggregated_rank'] = trackToRank[track]
        if pd.notna(trackRow[track].image_link):
            result[year][country][rank_key]['image'] = trackRow[track].image_link
        else :
            result[year][country][rank_key]['image'] = None

with open('top10_track_by_country.json', 'w') as f:
    json.dump(result, f, indent=2, )

In [11]:
"['Rimas Entertainment LLC.']".replace("[", "").split("'")

['', 'Rimas Entertainment LLC.', ']']